# NiN，网络中的网络

AlexNet和VGG在设计上的共同之处：

+ 首先使用卷积层构成的模块，充分抽取空间特征，

+ 然后使用全连接层构成的模块，输出分类结果。

> 相比于LeNet，AlexNet和VGG改进主要在于模块变得更宽(增加通道数)，网络变得更深。


Note:

+ 卷积层的输入和输出是四维数组：(样本，通道，高，宽)， 

+ 全连接层的输入和输出是二维数组：(样本，特征)， 

+ 如果在全连接层后面接一个卷积层，则需要变换维度，

+ 1\*1的卷积层，等价于全连接层：高和宽上的每个元素，相当于一个样本，通道相当于特征。

NiN在卷积层之后，使用1\*1的卷积层作为全连接层。卷积层和1\*1的卷积层，共同构成一个块。NIN网络就是由这样的块所组成的。

NiN块，代码：

In [ ]:
import time 
import torch
from torch import nn,optim 

import d2l_pytorch as d2l 
import sys 
sys.path.append('..')

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def nin_block(in_channels,out_channels,kernel_size,stride,padding):
    blk=nn.Sequential(
        nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding),
        nn.ReLU(),
        nn.Conv2d(out_channels,out_channels,kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels,out_channels,kernel_size=1),
        nn.ReLU()
    )
    return blk 

## NiN模型

NiN使用的卷积窗口分别为：11\*11,5\*5,3\*3，输出通道与AlexNet一致。

每个blk后面接一个步幅为2，窗口大小为3\*3的池化层。

相比于AlexNet，NiN去掉了最后的3个全连接层，取而代之的是：输出通道为标签个数的NiN块。最后，网络使用全局平均池化层，对每个通道中所有元素求平均，以直接用来分类。
> 全局平均池化层，就是窗口形状等于输入的平均池化层。

输出通道为标签个数的NiN块代替全脸基层的好处是：能防止过拟合。
> 全连接输出层，容易造成过拟合。



In [3]:
# 定义全局池化层

import torch.nn.functional as F 

class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d,self).__init__()
    def forward(self,x):
        # 全局池化层的大小，等于输入的高和宽
        return F.avg_pool2d(x,kernel_size=x.size()[2:])

In [4]:
def NiN():
    net=nn.Sequential(
        nin_block(1,96,kernel_size=11,stride=4,padding=0),
        nn.MaxPool2d(kernel_size=3,stride=2),
        nin_block(96,256,kernel_size=5,stride=1,padding=2),
        nn.MaxPool2d(kernel_size=3,stride=2),
        nin_block(256,384,kernel_size=3,stride=1,padding=1),
        nn.MaxPool2d(kernel_size=3,stride=2),
        nn.Dropout(0.5),
        # 10类
        nin_block(384,10,kernel_size=3,stride=1,padding=1),
        GlobalAvgPool2d(),
        # 将4维输出，转为2维输出，
        # 其大小为(批量，10)
        d2l.FlattenLayer()
    )
    return net

In [5]:
# test
net=NiN()
X=torch.rand(1,1,224,224)
for name,blk in net.named_children():
    X=blk(X)
    print(name,'output shape:',X.shape)

0 output shape: torch.Size([1, 96, 54, 54])
1 output shape: torch.Size([1, 96, 26, 26])
2 output shape: torch.Size([1, 256, 26, 26])
3 output shape: torch.Size([1, 256, 12, 12])
4 output shape: torch.Size([1, 384, 12, 12])
5 output shape: torch.Size([1, 384, 5, 5])
6 output shape: torch.Size([1, 384, 5, 5])
7 output shape: torch.Size([1, 10, 5, 5])
8 output shape: torch.Size([1, 10, 1, 1])
9 output shape: torch.Size([1, 10])


In [6]:
# 获取数据
batch_size=32
# 60000,10000
train_iter,test_iter=d2l.load_data_fashion_mnist_ch05(batch_size,resize=224)


In [7]:
print(len(train_iter))

1875


In [8]:
# 设置超参数
lr,num_epochs=0.002,5

optimizer=torch.optim.Adam(net.parameters(),lr=lr) 

In [ ]:
# 开始训练
d2l.train_ch05(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cuda
epoch 0/5, iter 0/1875, loss 2.308
epoch 0/5, iter 1/1875, loss 2.340
epoch 0/5, iter 2/1875, loss 2.300
epoch 0/5, iter 3/1875, loss 2.322
epoch 0/5, iter 4/1875, loss 2.327
epoch 0/5, iter 5/1875, loss 2.310
epoch 0/5, iter 6/1875, loss 2.278
epoch 0/5, iter 7/1875, loss 2.293
epoch 0/5, iter 8/1875, loss 2.309
epoch 0/5, iter 9/1875, loss 2.321
epoch 0/5, iter 10/1875, loss 2.308
epoch 0/5, iter 11/1875, loss 2.289
epoch 0/5, iter 12/1875, loss 2.302
epoch 0/5, iter 13/1875, loss 2.337
epoch 0/5, iter 14/1875, loss 2.325
epoch 0/5, iter 15/1875, loss 2.311
epoch 0/5, iter 16/1875, loss 2.329
epoch 0/5, iter 17/1875, loss 2.317
epoch 0/5, iter 18/1875, loss 2.284
epoch 0/5, iter 19/1875, loss 2.324
epoch 0/5, iter 20/1875, loss 2.305
epoch 0/5, iter 21/1875, loss 2.303
epoch 0/5, iter 22/1875, loss 2.291
epoch 0/5, iter 23/1875, loss 2.333
epoch 0/5, iter 24/1875, loss 2.335
epoch 0/5, iter 25/1875, loss 2.327
epoch 0/5, iter 26/1875, loss 2.290
epoch 0/5, iter 27/1